Устанавливаем необходимые библиотеки, а именно:
- `CSV` для загрузки данных из файла в формате csv
- `DataFrame` для обработки табличных данных, нужна чтобы обработать данные, которые мы загрузили из файла
- `NearestNeighborModels` чтобы использовать алгоритм обучения KNN
- `MLJ` для, непосредственно, обучения модели

In [ ]:
using Pkg

Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("NearestNeighborModels")
Pkg.add("MLJ")

Подключаем нужные библиотеки

In [ ]:
using CSV, DataFrames, MLJ, NearestNeighborModels

Загружаем датасет из файла

In [ ]:
df = CSV.read("./dataset.csv", DataFrame)

Выбираем все столбцы, кроме столбца с именем "variety" в качестве входных параметров. И затем преобразовываем их в матрицу

In [ ]:
X = Matrix(df[:, Not(:variety)])

Выбираем столбец variety в качестве классификаций

In [ ]:
Y = coerce(df.variety, Multiclass)

Разбиваем датасет на две выборки: для обучения и для тестирования. И затем приводим данные в формат, понятный библиотеке MLJ

In [ ]:
train_indices, test_indicies = partition(1:nrows(df), 0.7, shuffle=true)

X_train, X_test = selectrows(X, train_indices), selectrows(X, test_indicies)
Y_train, Y_test = Y[train_indices], Y[test_indicies]

X_train, X_test = MLJ.table(X_train), MLJ.table(X_test)

Инициализируем алгоритм KNN, указывая `8` в качестве числа ближайших соседей

In [ ]:
model = KNNClassifier(K=8)

Инициализируем модель и обучаем её на подготовленных данных

In [ ]:
mach = machine(model, X_train, Y_train)
MLJ.fit!(mach, verbosity=0)

Проверяем модель на тренировочной выборке и выводим её точность.

In [ ]:
predictions = mode.(MLJ.predict(mach, X_test))

acc = accuracy(predictions, Y_test)

println("Точность: ", round(acc * 100, digits=1), "%")